<a href="https://colab.research.google.com/github/lukerupolo/scorecard_generator/blob/main/event_marketing_colab_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Cell 1: Install all required packages and configure your ngrok token
!pip install -q streamlit pyngrok pandas openpyxl

from pyngrok import ngrok, conf
# Cell 1: Install all required packages and configure your ngrok token
!pip install -q streamlit pyngrok pandas openpyxl

from pyngrok import ngrok, conf

# 🔑 Replace this with your actual ngrok Authtoken:
!ngrok config add-authtoken 2xyaMUyxj5KM0l5tTL4evh59QfJ_7pmCiswiRKF93Hp3xGZcn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.0 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
%%writefile event_marketing_app.py
# -------------------------------------------------------------
# Streamlit – Event Marketing Analytics Suite
# Modes:
# 1) Generate template       → configure events & download blank workbook
# 2) Final benchmarks        → upload completed workbook & download summaries
# -------------------------------------------------------------
from __future__ import annotations

import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from io import BytesIO

# ---------------------------------------------------------
# Helpers
# ---------------------------------------------------------
def format_span_labels(event_date: datetime) -> tuple[str, str]:
    b_start = event_date - timedelta(days=7)
    b_end   = event_date - timedelta(days=1)
    a_end   = event_date + timedelta(days=6)
    baseline = f"Baseline {b_start:%Y-%m-%d} → {b_end:%Y-%m-%d}"
    actual   = f"Actual  {event_date:%Y-%m-%d} → {a_end:%Y-%m-%d}"
    return baseline, actual

def generate_event_tables(events: list[dict], metrics: list[str], countries: list[str]) -> dict[str, pd.DataFrame]:
    sheets: dict[str, pd.DataFrame] = {}
    for ev in events:
        baseline_col, actual_col = format_span_labels(ev["date"])
        template = pd.DataFrame({
            "Metric":          metrics,
            baseline_col:       [None] * len(metrics),
            actual_col:         [None] * len(metrics),
            "Baseline Method": [None] * len(metrics),
        })
        for country in countries:
            name = ev["name"][:25] or "Event"
            sheets[f"{name}_{country}"] = template.copy()
    return sheets

def compute_final_benchmarks(uploaded_file) -> dict[str, pd.DataFrame]:
    xls = pd.ExcelFile(uploaded_file)
    sheets_all = {sh: pd.read_excel(xls, sh) for sh in xls.sheet_names}
    region_groups: dict[str, list[pd.DataFrame]] = {}

    for name, df in sheets_all.items():
        parts = name.rsplit("_", 1)
        if len(parts) != 2:
            continue
        region_groups.setdefault(parts[1], []).append(df)

    summary: dict[str, pd.DataFrame] = {}
    for region, dfs in region_groups.items():
        metrics_list = dfs[0]["Metric"].tolist()
        data = {"Metric": metrics_list}
        avg_actuals, avg_uplift, avg_method = [], [], []

        for m in metrics_list:
            acts, upls, meths = [], [], []
            for df in dfs:
                cols = df.columns.tolist()
                b_col, a_col = cols[1], cols[2]
                meth_col = cols[3] if len(cols) > 3 else None
                row = df[df["Metric"] == m]
                if row.empty:
                    continue
                base = row[b_col].iat[0]
                act  = row[a_col].iat[0]
                acts.append(act)
                if meth_col:
                    meths.append(row[meth_col].iat[0])
                if pd.notna(base) and pd.notna(act):
                    upls.append((act - base) / base * 100)

            avg_actuals.append(np.nanmean(acts) if acts else np.nan)
            avg_method.append(np.nanmean(meths) if meths else np.nan)
            avg_uplift.append(np.nanmean(upls) if upls else np.nan)

        data["Average Actuals"]        = avg_actuals
        data["Baseline Method"]        = avg_method
        data["Baseline Uplift Expect"] = avg_uplift
        data["Proposed Benchmark"]     = [np.nanmedian([a, b]) for a, b in zip(avg_actuals, avg_method)]
        summary[region] = pd.DataFrame(data)

    return summary

# ---------------------------------------------------------
# Streamlit App
# ---------------------------------------------------------
st.set_page_config(page_title="Event Marketing Analytics", layout="wide")
st.markdown(
    """
# 📊 Event Marketing Analytics Suite

Welcome! This tool helps you:

1. **Generate** a template workbook to track multiple events and regions.
2. **Download** the blank workbook, fill in your data (Baseline, Actual, Baseline Method).
3. **Upload** the completed workbook to compute final benchmarks per region.

Select a mode below to get started.
""",
    unsafe_allow_html=True
)

mode = st.sidebar.radio(
    "Select an action:",
    ["Generate template", "Final benchmarks"]
)

if mode == "Generate template":
    st.sidebar.header("Step 1: Configure template")
    st.sidebar.markdown(
        """
        Specify your event names, start dates, metrics, and regions.
        When ready, click **Generate template** to download the Excel file.
        """
    )
    n_events = st.sidebar.number_input("Quantity of events", 1, 20, 1)
    events: list[dict] = []
    for i in range(n_events):
        st.sidebar.subheader(f"Event {i+1}")
        nm = st.sidebar.text_input("Name", key=f"nm{i}") or f"Event{i+1}"
        dt = st.sidebar.date_input("Start date (T)", key=f"dt{i}")
        if isinstance(dt, list):
            dt = dt[0]
        events.append({"name": nm, "date": datetime.combine(dt, datetime.min.time())})
    metrics = st.sidebar.multiselect(
        "Metrics to measure:",
        [
            "Sessions", "DAU", "Revenue", "Installs", "Retention", "Watch Time", "ARPU", "Conversions",
            "Video Views (VOD)", "Hours Watched (Streams)", "Social Mentions", "Search Index",
            "Broadcast TWT", "PCCV", "AMA"
        ],
        default=["Sessions", "DAU", "Revenue", "Installs"]
    )
    countries = st.sidebar.multiselect(
        "Regions/Countries:",
        ["US","GB","KR","JP","BR","DE","AU","CA","FR","IN","TH","ID","SA"],
        default=["US","GB","AU"]
    )
    if st.sidebar.button("Generate template 📥"):
        sheets = generate_event_tables(events, metrics, countries)
        st.subheader("Preview of generated sheets (first 3)")
        for i, (nm, df) in enumerate(sheets.items()):
            if i == 3:
                st.caption(f"...and {len(sheets)-3} more sheets")
                break
            st.markdown(f"**{nm}**")
            st.dataframe(df)
        buffer = BytesIO()
        with pd.ExcelWriter(buffer, engine="openpyxl") as w:
            for nm, df in sheets.items():
                df.to_excel(w, index=False, sheet_name=nm)
        buffer.seek(0)
        st.download_button(
            "📥 Download blank template",
            buffer,
            "event_template.xlsx",
            "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )

else:
    st.sidebar.header("Step 2: Compute final benchmarks")
    st.sidebar.markdown(
        """
        Upload the completed template workbook (with Baseline, Actual, Baseline Method).
        The tool will compute Uplift%, average across events per region, and propose benchmarks.
        """
    )
    uploaded = st.file_uploader("Upload filled template Excel file", type="xlsx")
    if uploaded:
        summaries = compute_final_benchmarks(uploaded)
        st.subheader("Summary tables by region")
        for region, df in summaries.items():
            st.markdown(f"### {region}")
            st.dataframe(df)
        out = BytesIO()
        with pd.ExcelWriter(out, engine="openpyxl") as w:
            for region, df in summaries.items():
                df.to_excel(w, index=False, sheet_name=region)
        out.seek(0)
        st.download_button(
            "📥 Download final benchmarks",
            out,
            "final_benchmarks.xlsx",
            "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )


Writing event_marketing_app.py


In [4]:
import os, time
from pyngrok import ngrok

# ── Step 1: Kill any existing Streamlit processes ────────────
os.system("pkill -f streamlit")
time.sleep(1)

# ── Step 2: Start Streamlit in the background via python -m ────
cmd = (
    "nohup python3 -m streamlit run event_marketing_app.py "
    "--server.port 8501 "
    "--server.headless true "
    "--server.address 0.0.0.0 "
    "--server.enableCORS false "
    "--server.enableXsrfProtection false "
    "> streamlit.log 2>&1 &"
)
os.system(cmd)
time.sleep(5)  # wait for Streamlit to spin up

# ── Optional debug prints ─────────────────────────────────────
print("❓ Streamlit processes:")
os.system("ps -ax | grep '[s]treamlit' || echo '→ none'")

print("\n❓ Ports listening on 8501:")
os.system("netstat -nlp 2>/dev/null | grep 8501 || echo '→ nothing listening'")

print("\n📄 First 10 lines of streamlit.log:")
os.system("head -n 10 streamlit.log || echo '→ streamlit.log missing'")

# ── Step 3: Open (or reuse) a single ngrok tunnel ─────────────
tunnels = ngrok.get_tunnels()
if tunnels:
    public_url = tunnels[0].public_url
else:
    public_url = ngrok.connect(8501, bind_tls=True).public_url

print(f"\n🌐 Your Streamlit app is live at: {public_url}")


❓ Streamlit processes:

❓ Ports listening on 8501:

📄 First 10 lines of streamlit.log:

🌐 Your Streamlit app is live at: https://15d3-34-53-39-133.ngrok-free.app


In [6]:
%%writefile requirements.txt
        streamlit==1.35.0
        pandas==2.2.2
        openpyxl==3.1.2
        numpy==1.26.4

Overwriting requirements.txt
